In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [4]:
## Read the pdfs from folder
loader = PyPDFDirectoryLoader("./us_census")

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents = text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0})

In [5]:
len(final_documents)

316

In [10]:
# Embeddings using HuggingFace
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name = "BAAI/bge-small-en-v1.5",
    model_kwargs = {'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

c:\Users\Divyanshu Shukla\Desktop\projects\groq\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Divyanshu Shukla\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Divyanshu Shukla\Desktop\projects\groq\.venv\Lib\site-packages\huggi

In [16]:
import numpy as np
np.array(huggingface_embeddings.embed_query(documents[0].page_content))

array([-7.04288855e-02, -1.23842554e-02, -2.66667102e-02,  4.11087461e-02,
        6.13234006e-02,  4.28758524e-02, -3.12031247e-02,  1.08945612e-02,
       -9.10003558e-02, -3.86431366e-02,  5.13605997e-02,  4.69753593e-02,
       -3.68075371e-02, -1.93688013e-02,  1.25593995e-03,  1.43930232e-02,
       -3.27467383e-03, -1.73090789e-02, -2.35300399e-02,  4.39519919e-02,
       -3.57484818e-02,  5.30993156e-02, -4.95760702e-03, -3.12633887e-02,
        2.02355124e-02,  2.55952366e-02,  9.30069294e-03, -1.56416651e-03,
       -5.96426316e-02, -1.50115237e-01, -2.34086649e-03,  2.31445264e-02,
       -6.52164295e-02, -2.72858068e-02,  5.73886139e-03,  1.56529583e-02,
       -2.15224996e-02,  8.06813985e-02,  2.76255663e-02,  3.61731350e-02,
       -3.71296443e-02,  1.15361623e-02, -4.46854671e-03, -7.02856295e-03,
       -4.28288169e-02, -8.35747982e-04, -2.72072032e-02, -1.33522623e-03,
        1.39592998e-02, -3.40681337e-02,  3.92966047e-02, -1.98309496e-02,
        3.44090946e-02,  

In [19]:
np.array(huggingface_embeddings.embed_query(documents[0].page_content)).shape

(384,)

In [20]:
vector_store = FAISS.from_documents(final_documents[:120],HuggingFaceBgeEmbeddings())

c:\Users\Divyanshu Shukla\Desktop\projects\groq\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Divyanshu Shukla\.cache\huggingface\hub\models--BAAI--bge-large-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Divyanshu Shukla\Desktop\projects\groq\.venv\Lib\site-packages\huggingfac

In [22]:
query = "WHAT IS HEALTH INSURANCE COVERAGE ?"
relevant_documents = vector_store.similarity_search(query)

print(relevant_documents[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [23]:
retriever = vector_store.as_retriever(search_type="similarity",search_karwgs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F36C214C10>


In [80]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="facebook/opt-125m",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

c:\Users\Divyanshu Shukla\Desktop\projects\groq\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Divyanshu Shukla\.cache\huggingface\hub\models--facebook--opt-125m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Divyanshu Shukla\Desktop\projects\groq\.venv\Lib\site-packages\huggingfac

'What is the health insurance coverage?\n\nHealth insurance coverage is a type of insurance that is offered by a health insurance company. It is a type of insurance that is offered by a health insurance company. It is a type of insurance that is offered by a health insurance company. It is a type of insurance that is offered by a health insurance company. It is a type of insurance that is offered by a health insurance company. It is a type of insurance that is offered by a health insurance company. It is a type of insurance that is offered by a health insurance company. It is a type of insurance that is offered by a health insurance company. It is a type of insurance that is offered by a health insurance company. It is a type of insurance that is offered by a health insurance company. It is a type of insurance that is offered by a health insurance company. It is a type of insurance that is offered by a health insurance company. It is a type of insurance that is offered by a health insu

In [81]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [82]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])


In [83]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [84]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [85]:
result = retrievalQA.invoke({"query":query})
print(result['result'])

c:\Users\Divyanshu Shukla\Desktop\projects\groq\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

decreases (1.8 percentage points) 
in the uninsured rate from 2021 to 
2022, which is consistent with the 
decrease in the uninsured rates 
in both South Carolina and North 
Carolina.36 The uninsured rate in 14 
metropolitan areas did not statisti -
cally change between 2021 and 
2022.
34 These most populous metropolitan 
areas had the highest uninsured rates in 
2022 and 2021. Refer to < www.census.
gov/content/dam/Census/library/
publications/2022/acs/acsbr-013.pdf >.
35 When comparing metropolitan areas 
from 2021 to 2022, we used the 25 most 
populous metropolitan areas in 2022 as 
the basis for comparison. However, none of 
the 25 most populous metropolitan areas 
changed from 2021 to 2022; only the order 
of their population ranking within the 25 
metro areas did.
36 The changes in the uninsured 
rates from 2021 to 2022 in Dallas-Fort 
Worth-Arlington, TX 